<a href="https://colab.research.google.com/github/kartik-5479/Gym_management_system/blob/main/GYM_MANAGEMENT_SYSTEM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# ===== Gym app + cloudflared (fixed) =====
# Paste & run in Google Colab. This frees the port, starts Flask on port 8000,
# runs cloudflared, and reliably prints the trycloudflare public URL.

!pip install -q flask

import os, csv, hashlib, hmac, re, threading, time, subprocess
from datetime import datetime, timezone, timedelta
from flask import Flask, request, redirect, url_for, render_template_string, flash, send_file

# ---------- Config ----------
PORT = 8000
DATA_DIR = '/content/gym_data'
MEMBERS_FILE = os.path.join(DATA_DIR, 'members.csv')
ATTENDANCE_FILE = os.path.join(DATA_DIR, 'attendance.csv')

HASH_NAME = 'sha256'
ITERATIONS = 100_000
SALT_PREFIX = b'GYM-SALT-'
IST = timezone(timedelta(hours=5, minutes=30))

# ---------- Helper: free port ----------
print(f"Checking and freeing port {PORT} (if used)...")
try:
    # show processes using the port
    proc = subprocess.run(["/bin/sh", "-c", f"lsof -i :{PORT} -Pn -sTCP:LISTEN || true"], capture_output=True, text=True)
    out = proc.stdout.strip()
    if out:
        print("Processes listening on port", PORT, ":\n", out)
        print("Attempting to kill processes using port", PORT, "...")
        subprocess.run(["/bin/sh", "-c", f"fuser -k {PORT}/tcp || true"], check=False)
        time.sleep(1)
        print("Killed processes (if any).")
    else:
        print("No process listening on port", PORT)
except Exception as e:
    print("Could not check/kill port:", e)

# ---------- Data functions & init ----------
def ensure_data_dir():
    os.makedirs(DATA_DIR, exist_ok=True)

def _clean_phone(phone: str) -> str:
    return re.sub(r"\D", "", phone)

def readable_member_id(name: str, phone: str, dob: str) -> str:
    parts = [p for p in name.strip().split() if p]
    initials = 'X' if not parts else ''.join([p[0] for p in parts[:3]]).upper()
    digits = _clean_phone(phone)
    last4 = digits[-4:] if len(digits) >= 4 else digits.zfill(4)
    try:
        dt = datetime.strptime(dob.strip(), '%Y-%m-%d')
        dob_part = dt.strftime('%y%m%d')
    except Exception:
        digits_dob = re.sub(r"\D", "", dob)
        dob_part = digits_dob[-6:].zfill(6)
    return f"{initials}-{last4}-{dob_part}"

def hash_password(password: str, salt_material: str) -> str:
    salt = SALT_PREFIX + salt_material.encode('utf-8')
    dk = hashlib.pbkdf2_hmac(HASH_NAME, password.encode('utf-8'), salt, ITERATIONS)
    return dk.hex()

def verify_password(stored_hash: str, password_attempt: str, salt_material: str) -> bool:
    attempt_hash = hash_password(password_attempt, salt_material)
    return hmac.compare_digest(stored_hash, attempt_hash)

def init_files():
    ensure_data_dir()
    if not os.path.exists(MEMBERS_FILE):
        with open(MEMBERS_FILE, 'w', newline='', encoding='utf-8') as f:
            csv.writer(f).writerow(['id','name','phone','dob','email','password_hash','joined_on'])
    if not os.path.exists(ATTENDANCE_FILE):
        with open(ATTENDANCE_FILE, 'w', newline='', encoding='utf-8') as f:
            csv.writer(f).writerow(['timestamp','member_id','name','entry_type','notes'])

def read_members() -> dict:
    d = {}
    if not os.path.exists(MEMBERS_FILE):
        return d
    with open(MEMBERS_FILE, 'r', encoding='utf-8') as f:
        for row in csv.DictReader(f):
            d[row['id']] = row
    return d

def add_member(name: str, phone: str, dob: str, email: str, password: str) -> str:
    member_id = readable_member_id(name, phone, dob)
    joined_on = datetime.now(IST).isoformat()
    password_hash = hash_password(password, member_id)
    members = read_members()
    if member_id in members:
        existing = members[member_id]
        if existing['name'].strip().lower() == name.strip().lower() and _clean_phone(existing['phone']) == _clean_phone(phone):
            members[member_id].update({'name': name, 'phone': phone, 'dob': dob, 'email': email})
            with open(MEMBERS_FILE, 'w', newline='', encoding='utf-8') as f:
                w = csv.writer(f); w.writerow(['id','name','phone','dob','email','password_hash','joined_on'])
                for mid, info in members.items():
                    w.writerow([mid, info.get('name',''), info.get('phone',''), info.get('dob',''), info.get('email',''), info.get('password_hash',''), info.get('joined_on','')])
            return member_id
        import hashlib as _hl
        suffix = _hl.sha256((name+phone+dob).encode('utf-8')).hexdigest()[-3:].upper()
        member_id = f"{member_id}-{suffix}"
    ensure_data_dir()
    with open(MEMBERS_FILE, 'a', newline='', encoding='utf-8') as f:
        csv.writer(f).writerow([member_id, name, phone, dob, email, password_hash, joined_on])
    return member_id

def log_attendance(member_id: str, name: str, entry_type: str='IN', notes: str=''):
    ts = datetime.now(IST).isoformat()
    ensure_data_dir()
    with open(ATTENDANCE_FILE, 'a', newline='', encoding='utf-8') as f:
        csv.writer(f).writerow([ts, member_id, name, entry_type, notes])

def read_attendance() -> list:
    if not os.path.exists(ATTENDANCE_FILE):
        return []
    with open(ATTENDANCE_FILE, 'r', encoding='utf-8') as f:
        return list(csv.DictReader(f))

init_files()

# ---------- Flask app ----------
app = Flask(__name__)
app.secret_key = 'dev-key-for-demo'

HOME_HTML = """<!doctype html>
<h1>Gym Attendance Portal (Flask)</h1>
<p>Timestamps shown are in IST (UTC+5:30).</p>
<ul>
<li><a href="/register">Register Member</a></li>
<li><a href="/login">Member Entry (Login)</a></li>
<li><a href="/exit">Member Exit</a></li>
<li><a href="/members">View Members</a></li>
<li><a href="/attendance">View Attendance</a></li>
<li><a href="/export/members">Download Members CSV</a></li>
<li><a href="/export/attendance">Download Attendance CSV</a></li>
</ul>"""

REGISTER_HTML = """<h2>Register New Member</h2>
<form method="post">Full name:<br><input name="name" required><br><br>Phone (digits):<br><input name="phone" required><br><br>DOB (YYYY-MM-DD):<br><input name="dob" required><br><br>Email (optional):<br><input name="email"><br><br>Password:<br><input name="password" type="password" required><br><br><button type="submit">Register</button></form><p><a href="/">Back</a></p>"""

LOGIN_HTML = """<h2>Member Entry (Login)</h2>
<form method="post">Member ID:<br><input name="member_id" required><br><br>Password:<br><input name="password" type="password" required><br><br><button type="submit">Enter (Log IN)</button></form><p><a href="/">Back</a></p>"""

EXIT_HTML = """<h2>Member Exit</h2>
<form method="post">Member ID:<br><input name="member_id" required><br><br>Password:<br><input name="password" type="password" required><br><br><button type="submit">Exit (Log OUT)</button></form><p><a href="/">Back</a></p>"""

TABLE_HTML = """<h2>{{title}}</h2>
<table border="1" cellpadding="6" cellspacing="0"><thead><tr>{% for h in headers %}<th>{{h}}</th>{% endfor %}</tr></thead><tbody>{% for row in rows %}<tr>{% for h in headers %}<td>{{row[h]}}</td>{% endfor %}</tr>{% endfor %}</tbody></table><p><a href="/">Back</a></p>"""

@app.route('/')
def home(): return HOME_HTML

@app.route('/register', methods=['GET','POST'])
def register():
    if request.method == 'GET': return REGISTER_HTML
    name = request.form.get('name','').strip()
    phone = request.form.get('phone','').strip()
    dob = request.form.get('dob','').strip()
    email = request.form.get('email','').strip()
    password = request.form.get('password','').strip()
    if not (name and phone and dob and password):
        flash("Please fill required fields")
        return redirect(url_for('register'))
    mid = add_member(name, phone, dob, email, password)
    flash(f"Registered {name} with ID: {mid}")
    return redirect(url_for('register'))

@app.route('/login', methods=['GET','POST'])
def login():
    if request.method == 'GET': return LOGIN_HTML
    mid = request.form.get('member_id','').strip().upper()
    password = request.form.get('password','').strip()
    members = read_members()
    if mid not in members:
        flash("Member ID not found")
        return redirect(url_for('login'))
    if verify_password(members[mid]['password_hash'], password, mid):
        log_attendance(mid, members[mid]['name'], 'IN', notes='Web-entry')
        flash(f"Entry logged for {members[mid]['name']} ({mid})")
    else:
        log_attendance(mid, members[mid]['name'], 'FAILED', notes='Bad password')
        flash("Authentication failed")
    return redirect(url_for('login'))

@app.route('/exit', methods=['GET','POST'])
def exit_member():
    if request.method == 'GET': return EXIT_HTML
    mid = request.form.get('member_id','').strip().upper()
    password = request.form.get('password','').strip()
    members = read_members()
    if mid not in members:
        flash("Member ID not found")
        return redirect(url_for('exit_member'))
    if verify_password(members[mid]['password_hash'], password, mid):
        log_attendance(mid, members[mid]['name'], 'OUT', notes='Web-exit')
        flash(f"Exit logged for {members[mid]['name']} ({mid})")
    else:
        log_attendance(mid, members[mid]['name'], 'FAILED_EXIT', notes='Bad password')
        flash("Authentication failed")
    return redirect(url_for('exit_member'))

@app.route('/members')
def members_page():
    members = list(read_members().values())
    headers = ['id','name','phone','dob','email','joined_on']
    return render_template_string(TABLE_HTML, title="Registered Members", headers=headers, rows=members)

@app.route('/attendance')
def attendance_page():
    rows = list(reversed(read_attendance()))
    headers = ['timestamp','member_id','name','entry_type','notes']
    return render_template_string(TABLE_HTML, title="Attendance Log (latest first)", headers=headers, rows=rows)

@app.route('/export/members')
def export_members():
    ensure_data_dir()
    if not os.path.exists(MEMBERS_FILE):
        return "No members file found.", 404
    return send_file(MEMBERS_FILE, as_attachment=True, download_name='members.csv', mimetype='text/csv')

@app.route('/export/attendance')
def export_attendance():
    ensure_data_dir()
    if not os.path.exists(ATTENDANCE_FILE):
        return "No attendance file found.", 404
    return send_file(ATTENDANCE_FILE, as_attachment=True, download_name='attendance.csv', mimetype='text/csv')

# ---------- Start Flask in background ----------
def run_flask():
    app.run(host='0.0.0.0', port=PORT, debug=False, use_reloader=False)

threading.Thread(target=run_flask, daemon=True).start()
time.sleep(1)

# ---------- Download & start cloudflared ----------
cf_path = '/content/cloudflared'
if not os.path.exists(cf_path):
    print("Downloading cloudflared...")
    subprocess.run(["wget", "-q", "-O", cf_path, "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64"], check=False)
    subprocess.run(["chmod", "+x", cf_path], check=False)

print("Starting cloudflared tunnel; awaiting public URL...")
proc = subprocess.Popen([cf_path, "tunnel", "--url", f"http://localhost:{PORT}"], stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True)

public_url = None
# read stdout lines and find a trycloudflare URL (more reliable)
for line in proc.stdout:
    print(line, end="")
    if "trycloudflare.com" in line:
        m = re.search(r"https?://[^\s)]+trycloudflare\.com[^\s)]*", line)
        if m:
            public_url = m.group(0).strip().rstrip('",')
            break

if public_url:
    print("\nPublic URL:", public_url)
    print("Open the public URL to access your Flask app.")
else:
    print("\ncloudflared did not reveal a trycloudflare URL in the output above.")
    print("Check the printed output for clues (errors, rate-limits). If cloudflared fails, you can still access the app at http://localhost:%d (if running locally)." % PORT)


Checking and freeing port 8000 (if used)...
No process listening on port 8000
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit


Starting cloudflared tunnel; awaiting public URL...
2025-11-18T08:13:23Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-18T08:13:23Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-18T08:13:27Z INF +--------------------------------------------------------------------------------------------+
2025-11-18T08:13:27Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-18T08:13:27Z INF |  h